# ERA5 Explorer

In [1]:
from datetime import datetime
import os
import os.path
from pathlib import Path
import subprocess
import warnings

from IPython.display import clear_output, display, HTML, Markdown
import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

from ddsapi import Client

In [2]:
%matplotlib inline

In [3]:
# Request-related options

# dataset_options = [
#     ('ERA-5 Single Levels', 'era5-single-levels')
# ]

product_type_options = [
    ('Reanalysis', 'reanalysis')
]

variable_options = [
    ('2 m Dew Point Temperature', '2_metre_dewpoint_temperature'),
    ('Air Pressure at Mean Sea Level', 'air_pressure_at_mean_sea_level'),
    ('LWE Thickness of Snowfall Amount', 'lwe_thickness_of_snowfall_amount'),
    ('Surface Air Pressure', 'surface_air_pressure'),
    ('Surface Short Wave Flux', 'surface_downwelling_shortwave_flux_in_air'),
    ('Surface Thermal Radiation', 'surface_thermal_radiation_downwards'),
    ('2 m Temperature', '2_metre_temperature'),
    ('Total Precipitation', 'total_precipitation'),
    ('10 m Wind U-Component', '10_metre_u_wind_component'),
    ('10 m Wind V-Component', '10_metre_v_wind_component')
]

# latitude_bounds = {
#     'min': -90,
#     'max': 90
# }
# longitude_bounds = {
#     'min': 0,
#     'max': 360
# }

# start_date = {'year': 1979, 'month': 1, 'day': 1}
# stop_date = {'year': 2020, 'month': 12, 'day': 31}
start_date = {
    'year': 2020,
    'month': 1,
    'day': 1
}
stop_date = {
    'year': 2020,
    'month': 1,
    'day': 5
}

In [4]:
# Auxiliary data

_DIMENSIONS = {'time', 'latitude', 'longitude'}
data = {'dset': None, 'fpath': None, 'is_updated': False}

In [5]:
# Input and output widgets
out = widgets.Output()

intro = widgets.HTML(
    value='<b>Please, choose request-related options:</b>'
)
dds_api_key = widgets.Password(
    description='API Key:',
    placeholder='Please, enter DDS API key',
    layout=widgets.Layout(width='auto', height='auto')
)
# datasets = widgets.Dropdown(
#     options=dataset_options,
#     index=0,
#     disabled=False,
#     description='Dataset:',
#     layout=widgets.Layout(width='auto', height='auto')
# )
product_types = widgets.Dropdown(
    options=product_type_options,
    index=0,
    disabled=False,
    description='Product Type:',
    layout=widgets.Layout(width='auto', height='auto')
)
variables = widgets.SelectMultiple(
    options=variable_options,
    index=(0,),
    disabled=False,
    description='Variables:',
    rows=len(variable_options),
    layout=widgets.Layout(width='auto', height='auto')
)
area_grid = widgets.GridspecLayout(
    n_rows=6,
    n_columns=3,
    grid_gap='0px',
    width='auto',
    height='auto'
)
area_grid[0, 0] = widgets.HTML(
    value='Area bounds:',
    layout=widgets.Layout(width='auto', height='auto')
)
area_grid[0, 1] = widgets.HTML(
    value='North',
    layout=widgets.Layout(width='50%', height='auto')
)
area_grid[1, 1] = area_north = widgets.Text(
    value='48.0',
    placeholder='',
    description='',
    disabled=False,
    layout=widgets.Layout(width='50%', height='auto')
    
)
area_grid[2, 0] = widgets.HTML(
    value='West',
    layout=widgets.Layout(width='50%', height='auto')
)
area_grid[2, 2] = widgets.HTML(
    value='East',
    layout=widgets.Layout(width='50%', height='auto')
)
area_grid[3, 0] = area_west = widgets.Text(
    value='5.0',
    placeholder='',
    description='',
    disabled=False,
    layout=widgets.Layout(width='50%', height='auto')
    
)
area_grid[3, 2] = area_east = widgets.Text(
    value='18.0',
    placeholder='',
    description='',
    disabled=False,
    layout=widgets.Layout(width='50%', height='auto')
    
)
area_grid[4, 1] = widgets.HTML(
    value='South',
    layout=widgets.Layout(width='50%', height='auto')
)
area_grid[5, 1] = area_south = widgets.Text(
    value='35.0',
    placeholder='',
    description='',
    disabled=False,
    layout=widgets.Layout(width='50%', height='auto')
    
)
# latitudes = widgets.SelectionRangeSlider(
#     options=[
#         str(i)
#         for i in range(latitude_bounds['min'], latitude_bounds['max'] + 1)
#     ],
#     index=(125, 140),
#     disabled=False,
#     description='Latitude:',
#     orientation='horizontal',
#     readout=True,
#     continuous_update=False,
#     layout=widgets.Layout(width='auto', height='auto')
# )
# longitudes = widgets.SelectionRangeSlider(
#     options=[
#         str(i)
#         for i in range(longitude_bounds['min'], longitude_bounds['max'] + 1)
#     ],
#     index=(5, 18),
#     disabled=False,
#     description='Longitude:',
#     orientation='horizontal',
#     readout=True,
#     continuous_update=False,
#     layout=widgets.Layout(width='auto', height='auto')
# )
start_datetime = widgets.DatePicker(
    value=datetime(**start_date),
    disabled=False,
    description='Start Date:',
    layout=widgets.Layout(width='auto', height='auto')
)
stop_datetime = widgets.DatePicker(
    value=datetime(**stop_date),
    disabled=False,
    description='Stop Date:',
    layout=widgets.Layout(width='auto', height='auto')
)
run = widgets.Button(
    description='Retrieve',
    tooltip='Retrieve',
    icon='download',
    disabled=False,
    button_style='',
    layout=widgets.Layout(width='auto', height='auto')
)
time_indices = widgets.IntSlider(
    value=0,
    min=0,
    max=1,
    step=1,
    description='Plot Time:',
    disabled=True,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d',
    layout=widgets.Layout(width='auto', height='auto')
)
selected_variables = widgets.Dropdown(
    options=[],
    # index=0,
    disabled=True,
    description='Plot Variable:',
    layout=widgets.Layout(width='auto', height='auto')
)
link = widgets.HTML(
    value='Link currently not available',
    # placeholder='Link currently not available',
    description='Link:',
    layout=widgets.Layout(width='auto', height='auto')
)
in_box = widgets.VBox(
    children=[
        intro,
        dds_api_key,
        # datasets,
        product_types,
        variables,
        area_grid,
        # latitudes,
        # longitudes,
        start_datetime,
        stop_datetime,
        run
    ],
    layout=widgets.Layout(width='auto', height='auto')
)
out_box = widgets.VBox(
    children=[
        out,
        time_indices,
        selected_variables,
        link
    ],
    layout=widgets.Layout(width='auto', height='auto')
)
form = widgets.TwoByTwoLayout(
    top_left=in_box,
    top_right=out_box,
    grid_gap='0px'
)

In [6]:
# Required functions

def plot(time_index=0, variable=None):
    if not variable:
        variable = selected_variables.options[0]
    darr = data['dset'][variable].isel(indexers={'time': time_index})
    darr.plot.pcolormesh()
    plt.show()


def display_widgets():
    display(form)


def retrieve():
    api_key = dds_api_key.value
    dataset = 'era5-single-levels'  # datasets.value
    request = {
        'product_type': product_types.value,
        'variable': variables.value,
        'area': {
            'south': float(area_south.value),
            'north': float(area_north.value),
            'west': float(area_west.value),
            'east': float(area_east.value)
#             'south': float(latitudes.value[0]),
#             'north': float(latitudes.value[1]),
#             'west': float(longitudes.value[0]),
#             'east': float(longitudes.value[1])
        },
        'time': {
            'start': start_datetime.value.isoformat(),
            'stop': stop_datetime.value.isoformat()
        },
        'format': 'netcdf'
    }
    data['fpath'] = file_path = f"{dataset}-{request['product_type']}.nc"

    client = (
        Client(quiet=True, url='https://ddsapi.cmcc.it/v1', key=api_key)
        if api_key else
        Client() 
    )
#     display(HTML(f'Retrieve initialized at {datetime.now().time()}.'))
    client.retrieve(dataset, request, file_path)
#     display(HTML(f'Retrieve completed at {datetime.now().time()}.'))

    # !clamscan -r --bell -i {os.getcwd()}
#     scan_result = subprocess.run(
#         ['clamscan', '-r', '--bell', '-i', os.getcwd()],
#         capture_output=True
#     )
#     print(scan_result.stdout.decode('UTF-8'))
#     if err := scan_result.stderr.decode('UTF-8'):
#         warnings.warn(
#             message=f'scan not completed:\n{err}',
#             category=RuntimeWarning
#         )


def reset():
    time_indices.value = 0
    time_indices.max = 1
    time_indices.disabled = True
    selected_variables.options = []
    selected_variables.disabled = True

    
def update_data():
    # NOTE: This is an embarrassingly dirty hack to avoid the problem due to
    # which `xarray.open_dataset` returns previously loaded (possibly cached)
    # dataset.
    with xr.open_dataset(data['fpath'], cache=False) as dset:
        pass
    with xr.open_dataset(data['fpath'], cache=False) as dset:
        data['dset'] = dset
        vars_ = sorted(list(set(dset.variables.keys()) - _DIMENSIONS))
        time_indices.value = 0
        time_indices.max = dset.coords['time'].size - 1
        time_indices.disabled = False
        selected_variables.options = vars_
        selected_variables.disabled = False


def update_link():
    if data['fpath']:
        path = os.path.join(*Path(os.getcwd()).parts[3:], data['fpath'])
        path = f'http://localhost:8888/lab/tree/{path}'
        link.value = f'<a href="{path}">Download data</a>'


def update_out(time_index, variable):
    print('update_out', 1, data['is_updated'])
    if data['is_updated']:
        with out:
            clear_output()
            plot(time_index, variable)


def run_on_click():
    data['is_updated'] = False
    reset()
    retrieve()
    update_data()
    update_link()
    out.clear_output()
    data['is_updated'] = True
    display_plot()


@out.capture(clear_output=True)
def display_plot(time_index=0, variable=None):
    plot(time_index, variable)

In [7]:
# Additional functionality related to widgets

interact = widgets.interactive(
    update_out,
    time_index=time_indices,
    variable=selected_variables
)

run.on_click(lambda run: run_on_click())

In [8]:
# Running

display_widgets()

TwoByTwoLayout(children=(VBox(children=(HTML(value='<b>Please, choose request-related options:</b>'), Password…